In [2]:
import pandas as pd
import numpy as np
import copy
import re
from nltk import WordNetLemmatizer

In [3]:
import os
import google.datalab as datalab
import google.datalab.ml as ml

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import re
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

In [6]:
storage_bucket = 'gs://staging.cpb100-200708.appspot.com/'
workspace_path = os.path.join(storage_bucket, 'data')

In [7]:
from io import BytesIO

%gcs read --object "gs://staging.cpb100-200708.appspot.com/data/train.csv" --variable csv_as_bytes

train = pd.read_csv(BytesIO(csv_as_bytes))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
from io import BytesIO
%gcs read --object "gs://staging.cpb100-200708.appspot.com/data/test.csv" --variable csv_as_bytes

test = pd.read_csv(BytesIO(csv_as_bytes))
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
def decontract(text):  # words=[APPO[word] if word in APPO else word for word in words]
    contract = {"'ll" : " will", "'d" : " would", "\n" : " ", "haha": "ha",
    "hahaha": "ha", "don't": "do not", "doesn't": "does not", "didn't": "did not",
    "hasn't": "has not", "haven't": "have not", "hadn't": "had not", "won't": "will not",
    "wouldn't": "would not", "can't": "can not", "cannot": "can not", "i'm": "i am",
    "i'll" : "i will", "it's" : "it is", "'s" : " is", "that's" : "that is", "weren't" : "were not",}
    for k in contract.keys():
        text = text.replace(k, contract[k])
    return text


def de_punct(text):
    return re.sub('[%s]+' % re.escape(string.punctuation), " ", text.strip())  # [^a-zA-Z ?!]+

def de_stop(text, stp_words_list):
    
    textl = text.split(" ")
    out = []
    for w in textl:
        if w not in stp_words_list:
            out.append(w)
            
    return " ".join(out)

def de_removeNum(text):
    textl = text.split(" ")
    alpha = []
    for item in textl:
        if item.isalpha():
            alpha.append(item)
    return " ".join(alpha)
     
    
def de_stem(text):
    textl = text.split(" ")
    #stemmer = PorterStemmer()
    stems = []
    stemmer = SnowballStemmer("english")
    for word in textl:
        stems.append(stemmer.stem(word))
    return " ".join(stems)

def de_unique(orig_text):
    unique_words = []
    for w in orig_text:
        if w not in unique_words:
            unique_words.append(w)
    return unique_words

In [ ]:
import nltk
from nltk.corpus import stopwords

add_list = ["could", "said", "would", "first", "get", "bye",] #one
stop_words = stopwords.words("english")
for word in add_list:
    if word not in stop_words:
        stop_words.append(word)
        
#print(stop_words)
def pre_processing(text):
    # lower
    text = text.lower()

    text = decontract(text)

    text = de_punct(text)
    
    text = de_stop(text, stop_words)
    
    text = de_removeNum(text)
    
    text = de_stem(text)
    
    #text = de_unique(text)
    
    return text
        
def main():
  
    processed_comments = list(map(pre_processing, train['comment_text']))
    process_test = list(map(pre_processing, test['comment_text']))
    train['comment_text'] = processed_comments
    test['comment_text'] = process_test
    train.to_csv("train_preprocessed.csv", index=False)
    test.to_csv("test_preprocessed.csv", index=False)
    
if __name__ == "__main__":
    main()